<a href="https://colab.research.google.com/github/librairy/bio-nlp/blob/master/notebooks/drugs/atc_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 

atc_data = pd.read_csv(
    "data/atc-codes.csv",      # relative python path to subdirectory
    sep=',',
    na_filter=False,
    quotechar="\"",        # single quote allowed as quote character
    usecols=['Class ID','Preferred Label','Synonyms','CUI','Parents','ATC LEVEL']
)

print(atc_data.shape)
atc_data.head()

(6358, 6)


,Class ID,Preferred Label,Synonyms,CUI,Parents,ATC LEVEL
0,http://purl.bioontology.org/ontology/UATC/A03AX13,silicones,,C0037114,http://purl.bioontology.org/ontology/UATC/A03AX,5
1,http://purl.bioontology.org/ontology/UATC/J01DB07,cefatrizine,,C0007545,http://purl.bioontology.org/ontology/UATC/J01DB,5
2,http://purl.bioontology.org/ontology/UATC/V09AA,"Technetium 99m compounds, central nervous syst...",Technetium (99mTc) compounds,C3541402,http://purl.bioontology.org/ontology/UATC/V09A,4
3,http://purl.bioontology.org/ontology/UATC/G03GA04,urofollitropin,,C0524639,http://purl.bioontology.org/ontology/UATC/G03GA,5
4,http://purl.bioontology.org/ontology/UATC/D04AA10,promethazine,,C0033405,http://purl.bioontology.org/ontology/UATC/D04AA,5


In [ ]:
import pysolr
import math

solr = pysolr.Solr('http://librairy.linkeddata.es/data/atc', timeout=2)

documents = []
num = 0;
for i in atc_data.itertuples(index=False):
    atc_document = {}
    if (len(i[0]) > 0 ):
        atc_document['id']= i[0].split("/")[-1]
    if (len(i[1]) > 0 ):
        atc_document['label_t']= i[1].lower()
    synonyms = []
    if (len(i[2]) > 0 ):
        synonyms.append(i[2].lower())
    atc_document['synonyms']= synonyms
    if (len(i[3]) > 0 ):
        atc_document['cui_s']= i[3]
    if (len(i[4]) > 0 ):
        atc_document['parent_s']= i[4].split("/")[-1]
    if (len(i[5]) > 0 ):
        atc_document['level_i']= i[5]    
    documents.append(atc_document) 
    num+=1
    if (len(documents) > 0 and len(documents) %100 == 0):
        solr.add(documents)
        solr.commit()
        documents = []
        print(num,"drugs added")

100 drugs added
200 drugs added
300 drugs added
400 drugs added
500 drugs added
600 drugs added
700 drugs added
800 drugs added
900 drugs added
1000 drugs added
1100 drugs added
1200 drugs added
1300 drugs added
1400 drugs added
1500 drugs added
1600 drugs added
1700 drugs added
1800 drugs added
1900 drugs added
2000 drugs added
2100 drugs added
2200 drugs added
2300 drugs added
2400 drugs added
2500 drugs added
2600 drugs added
2700 drugs added
2800 drugs added
2900 drugs added
3000 drugs added
3100 drugs added
3200 drugs added
3300 drugs added
3400 drugs added
3500 drugs added
3600 drugs added
3700 drugs added
3800 drugs added
3900 drugs added
4000 drugs added
4100 drugs added
4200 drugs added
4300 drugs added
4400 drugs added
4500 drugs added
4600 drugs added
4700 drugs added
4800 drugs added
4900 drugs added
5000 drugs added
5100 drugs added
5200 drugs added
5300 drugs added
5400 drugs added
5500 drugs added
5600 drugs added
5700 drugs added
5800 drugs added
5900 drugs added
6000 d

# Adding CIMA Codes

In [ ]:
drugs_data = pd.read_csv(
    "data/drugs.csv",      # relative python path to subdirectory
    sep=';',           # Tab-separated value file.
    quotechar="\"",        # single quote allowed as quote character
    usecols=['Nº Registro', 'Medicamento', 'Cód. ATC']
)

print(drugs_data.shape) 
drugs_data.head()



(23407, 3)


,Nº Registro,Medicamento,Cód. ATC
0,40537,INACID 25 mg CAPSULAS DURAS,M01AB01
1,7235,PASSIFLORINE SOLUCION ORAL,N05CM
2,18329,TIOBARBITAL BRAUN 1 G,N01AF03
3,33561,DIHYDERGOT COMPRIMIDOS,N02CA01
4,34369,GLUCOSADA GRIFOLS 5% SOLUCION PARA PERFUSION,B05BA03


In [ ]:
atc_regs = drugs_data.groupby('Cód. ATC')['Nº Registro'].apply(list).reset_index()
print(atc_regs)

     Cód. ATC                                       Nº Registro
0     A01AA01                                    [57745, 57746]
1       A01AB  [38791, 34484, 35528, 12912, 45563, 9957, 29898]
2     A01AB03                                    [84343, 80642]
3     A01AB09                                    [55969, 55962]
4     A01AB19                                           [39783]
...       ...                                               ...
2227    V10XX                              [05322001, 05322002]
2228  V10XX02                                        [03264001]
2229  V10XX03                                       [113873001]
2230  V10XX04                                      [1171226001]
2231   XXXXXX               [75668, 75670, 75659, 75661, 75663]

[2232 rows x 2 columns]


In [ ]:
import pysolr

solr = pysolr.Solr('http://librairy.linkeddata.es/data/atc', timeout=2)

def add_num_regs(atc_code,num_regs):
    solr_query = "id:"+str(atc_code)
    for result in solr.search(q=solr_query,rows=1):
        result['cima_codes']=num_regs
        del (result['_version_'])
        return result
    return {}

sample_doc = {
  "id": "1212121212112121212",
  "cima_codes":["a","b","c"]
}
solr.add([sample_doc])
solr.commit()
solr.delete(id=sample_doc['id'])
solr.commit()
updated_drugs = []
num = 0;
for i in atc_regs.itertuples():
    atc_code = i[1]
    updated_drugs.append(add_num_regs(atc_code,[str(code) for code in i[2]])) 
    num+=1
    if (len(updated_drugs) == 0):
        solr.add(updated_drugs)
        solr.commit()
        updated_drugs = []
        print(num,"drugs updated")

100 drugs updated
200 drugs updated
300 drugs updated
400 drugs updated
500 drugs updated
600 drugs updated
700 drugs updated
800 drugs updated
900 drugs updated
1000 drugs updated
1100 drugs updated
1200 drugs updated
1300 drugs updated
1400 drugs updated
1500 drugs updated
1600 drugs updated
1700 drugs updated
1800 drugs updated
1900 drugs updated
2000 drugs updated
2100 drugs updated
2200 drugs updated


# Adding medical names

In [ ]:
import pandas as pd

md_df = pd.read_excel('data/Medicines_output_european_public_assessment_reports.xlsx', skiprows=8, keep_default_na=False)
md_df.head()

,Category,Medicine name,Therapeutic area,International non-proprietary name (INN) / common name,Active substance,Product number,Patient safety,Authorisation status,ATC code,Additional monitoring,...,Vet pharmacotherapeutic group,Date of opinion,Decision date,Revision number,Condition / indication,Species,ATCvet code,First published,Revision date,URL
0,Human,Eliquis,"Arthroplasty, Venous Thromboembolism",apixaban,Apixaban,EMEA/H/C/002148,yes,Authorised,B01AF02,no,...,,,2020-03-26 01:00:00,20,For Eliquis 2.5 mg film-coated tablets:Prevent...,,,2018-08-03 16:27:00,2020-04-22 17:05:00,https://www.ema.europa.eu/en/medicines/human/E...
1,Human,Temozolomide Sun,"Glioma, Glioblastoma",temozolomide,temozolomide,EMEA/H/C/002198,no,Authorised,L01AX03,no,...,,,2020-03-11 01:00:00,16,Temozolomide Sun is indicated for the treatmen...,,,2018-08-03 00:00:00,2020-04-22 16:34:00,https://www.ema.europa.eu/en/medicines/human/E...
2,Human,Atazanavir Mylan,HIV Infections,atazanavir,atazanavir sulfate,EMEA/H/C/004048,no,Authorised,J05AE08,no,...,,,2020-03-12 01:00:00,8,"Atazanavir Mylan, co-administered with low dos...",,,2018-05-03 17:20:00,2020-04-22 16:00:00,https://www.ema.europa.eu/en/medicines/human/E...
3,Human,Vivanza,Erectile Dysfunction,vardenafil,vardenafil,EMEA/H/C/000488,no,Authorised,G04BE09,no,...,,,2020-03-19 01:00:00,27,Treatment of erectile dysfunction in adult men...,,,2018-04-19 00:00:00,2020-04-22 16:00:00,https://www.ema.europa.eu/en/medicines/human/E...
4,Human,Juluca,HIV Infections,dolutegravir / rilpivirine,"dolutegravir sodium, rilpivirine hydrochloride",EMEA/H/C/004427,yes,Authorised,J05AR,yes,...,,2018-03-22 01:00:00,2020-04-01 00:00:00,2,Juluca is indicated for the treatment of human...,,,2018-05-16 12:26:00,2020-04-22 15:44:00,https://www.ema.europa.eu/en/medicines/human/E...


In [ ]:
import pysolr
import html2text
import urllib.request

solr = pysolr.Solr('http://librairy.linkeddata.es/data/atc', timeout=20)

h = html2text.HTML2Text()
h.ignore_links = True

def add_field_in_list(resource,field,value):
    if (len(value) > 0 ):
        if (not field in resource):
            resource[field] = []
        resource[field].append(value)  

def get_txt(url):
    fp = urllib.request.urlopen(url)
    mybytes = fp.read()
    mystr = mybytes.decode("utf8")
    fp.close()
    return h.handle(mystr)

documents = []
i = 1
for row in md_df.itertuples():
    atc_code = row[9]
    if (len(atc_code) > 1):
        for drug in solr.search("id:"+atc_code):
            add_field_in_list(drug,'category',row[1])
            add_field_in_list(drug,'medicines',row[2])
            add_field_in_list(drug,'therapeutic_area',row[3])
            add_field_in_list(drug,'emea_codes',row[6])
            add_field_in_list(drug,'overviews',row[30])
            solr.add([drug])
            solr.commit()
            #print(drug)
            if (i % 100 == 0):
                print(i,"drugs updated")
            i+=1

100 drugs updated
200 drugs updated
300 drugs updated
400 drugs updated
500 drugs updated
600 drugs updated
700 drugs updated
800 drugs updated
900 drugs updated
1000 drugs updated
1100 drugs updated
1200 drugs updated
1300 drugs updated
